# 第8章 循环神经网络

## 8.1 序列模型

### 练习 8.1.1

改进本节实验中的模型。
1. 是否包含了过去$4$个以上的观测结果？真实值需要是多少个？
1. 如果没有噪音，需要多少个过去的观测结果？提示：把$\sin$和$\cos$写成微分方程。
1. 可以在保持特征总数不变的情况下合并旧的观察结果吗？这能提高正确度吗？为什么？
1. 改变神经网络架构并评估其性能。

**解答：** 

### 练习 8.1.2

一位投资者想要找到一种好的证券来购买。他查看过去的回报，以决定哪一种可能是表现良好的。这一策略可能会出什么问题呢？

**解答:**

该投资者仅采用过去的回报作为参考信息来预测未来的证券表现，属于外推法。
外推法在预测未来数据时，存在假设局限性，即数据必须满足其过去历史可以用来预测未来的趋势。如果假设不成立，那么预测结果很可能会不准确。此外，外推法不能预测突发事件的发生，并且预测结果非常依赖数据质量。如果历史数据不准确，预测结果也会收到影响。
对于金融证券来说，决定证券回报的因素不仅仅包括过去的回报曲线，还包括政治事件、行业趋势和金融危机等外部因素，而这些因素往往是决定证券表现的重要因素。因此在使用外推法预测证券未来回报时，很难考虑到随时变化的市场环境以及突发的外部因素，从而带来产生巨大误差。


### 练习 8.1.3

时间是向前推进的因果模型在多大程度上适用于文本呢？

**解答：**
时间是向前推进的因果模型是一种基于时间序列和因果关系的模型，该模型假设未来的结果受过去因素和时间序列影响，并且无法反过来影响过去因素。
但是在文本数据中，时间序列并非影响未来文本结果产生的唯一因素，并且某种程度上来讲，时序与文本内容的产生并无显著关联。
因此采用时间前向推进的因果模型并不能很好的适用于文本场景。需要通过建立更加复杂的模型来考虑影响文本内容的多种因素。

### 练习 8.1.4

举例说明什么时候可能需要隐变量自回归模型来捕捉数据的动力学模型。

**解答：**
隐变量自回归模型是一种用于建模时间序列数据中的动态变化的自回归模型，该模型通过引入一些隐变量来捕捉数据的动态性质。
动力学模型则是一种描述系统状态随时间变化而变化的数学模型，在该模型中，系统的状态可以用一组状态变量来表示，并且这些状态变量随着时间的推移而发生变化。
因此，在实际场景中，股票价格变化通常受时间序列和市场因素的影响，可通过隐变量自回归模型自回归的建模股票在时间上的依赖关系，并通过隐变量来捕捉未观测到的市场信息，最终建立起股票价格的动力学模型。

## 8.2 文本预处理 

### 练习 8.2.1

词元化是一个关键的预处理步骤，它因语言而异。尝试找到另外三种常用的词元化文本的方法。

**解答：**

三种常用的词元化文本的方法如下：

1. BPE(Byte-Pair Encoding)：字节对编码，该方法本质是一种贪心算法，具体流程如下：

   - 在语料库中单词末尾添加 <code></w></code>，并统计该单词出现的次数
   - 将单词切分为单个字符作为子词，并以此构建出初始子词词表
   - 在语料库中，统计单词内相邻子词对的频数
   - 将频数最高的子词对合并为新的子词，并加入到子词词表中
   - 重复上述两步，直到达到合并次数或子词词表中所有子词的频数均为1。

   通过该方法，对语料库实现了数据压缩，实现通过最少的token数表示一个corpus

2. WordPiece：WordPiece与BPE方法类似，本质也是一种贪心，但是不同于BPE选择出现频率最高的两个子词合并为新的子词，WordPiece选择具有较强关联性的子词进行合并。具体流程如下：

   - 将语料库中单词切分为单个字符作为初始化的子词词表，假设每个子词独立，此时语言模型似然值等价于子词概率积
   - 两两拼接子词，并统计新子词加入词表后对语言模型似然值的提升程度
   - 最终选择对语言模型似然度提升最大的字符加入到词表中
   - 重复上述两步，直到词表大小达到指定大小。

3. SentencePiece：不同于BPE和WordPiece中采用空格区分不同单词的方式，SentencePiece将空格也进行编码，不考虑句子中单独的单词，而是将整个句子作为整体进行拆分。再通过BPE或Unigram的方式构造词表。

### 练习 8.2.2

在本节的实验中，将文本词元为单词和更改`Vocab`实例的`min_freq`参数。这对词表大小有何影响？

**解答：**
词元设置为Char时，输出的语料库和词表大小为 $(170580, 28)$，结果如下：
![](../..//images/token_char.png)
而当词元设置为Word时，输出的语料库和词表大小为 $(32775, 4580)$，结果如下：
![](../../images/token_word.png)

在本节实验中，Vocab 实例的 **min_freq** 参数主要用来实现对低频词的过滤，默认 **min_freq=0**，结果如下图所示：
![](../../images/min_freq=0.png)
设置 **min_freq=10**，过滤掉低频词，所得对词表的影响结果如下图所示：
![](../../images/min_freq=10.png)

## 8.3 语言模型和数据集

### 练习 8.3.1

假设训练数据集中有$100,000$个单词。一个四元语法需要存储多少个词频和相邻多词频率？

**解答：**
对于N-Gram语法，其存储的词频数与单词个数以及N有关，假设词汇表中只存在三个单词“'早','上','好'”，则对于二元语法，词表内容为“‘早早’。‘早上’，‘早好’，‘上早’，‘上上’，‘上好’，‘好早’，‘好上’，‘好好’”，词频数为 $3^2=9$个，因此对于100000个单词的词表，其通过四元组存储词频数时，仅考虑相邻4个单词组成的长为4的子序列，词频数为 $(1 \times 10^5)^4=1 \times 10^{20}$。

而对于多词频率，在N-Gram语法中，会针对每个单词计算其相邻多词频率，因此对于训练集中的100000个单词，会存储100000个相邻多词频率。

### 练习 8.3.2

我们如何对一系列对话建模？

**解答:**
可以考虑将一系列对话拼接为一个长序列，通过对特殊分隔符对每个对话进行分隔。最后通过循环神经网络对长序列进行建模。

此外也可采用Seq2Seq模型，将对话看作一个序列进行建模，通过输入一段序列文本来生成新的序列文本。

### 练习 8.3.3

一元语法、二元语法和三元语法的齐普夫定律的指数是不一样的，能设法估计么？

**解答：**
齐普夫定律认为，对于大规模文本，将其中每个词的词频进行统计，并由高到低排序标号，则这些单词的频数$F$和这些单词的序号$R$之间存在一个常数$C$，满足$F \times R=C$，以"The Time Machine" 这篇文章为例，其各语法的齐普夫定律指数如下：
![](../../images/zipf_1.png)
![](../../images/zipf_2.png)

### 练习 8.3.4

想一想读取长序列数据的其他方法？

**解答：**
在读取长序列数据时，除了随机采样和顺序分区，还可以采用下列方法：
- 滑动窗口：通过滑动窗口，可以将一个长序列拆分为若干等长的有重叠子序列，在BERT中，这些子序列会被当做单独的文本进行处理，然后将所有子序列的输出结果进行整合，获得最终的长文本序列处理结果。
- 分层采样：将整个长序列先拆分为不同层次，使得每个层次包含不同的序列信息，通过这种方法，既可以保留长序列中的上下文信息，也可对整个长序列进行多层次的抽象概括。

### 练习 8.3.5

考虑一下我们用于读取长序列的随机偏移量。
1. 为什么随机偏移量是个好主意？
1. 它真的会在文档的序列上实现完美的均匀分布吗？
1. 要怎么做才能使分布更均匀？

**解答：**

1. 为什么随机偏移量是个好主意？
   如果固定偏移量，则会导致迭代后获取到的子序列仅覆盖有限的序列范围，无法保证覆盖性。
   ![](../../images/Un_random.png)
   加入随机偏移量后，每次迭代时从不同初始位置获取的子序列，既保证了覆盖性，也保证了随机性。
   ![](../../images/random.png)
2. 它真的会在文档的序列上实现完美的均匀分布吗？
   长序列采样时，通过为每个采样点加入随机偏移量可以在一定程度上增加子序列的覆盖性和随机性，但是并不能实现完美的均匀分布，因为加入偏移量后无法保证每个采样点的采样概率完全相等，偏移量的存在可能会使得某些子序列被频繁采样，某些子序列被跳过。
3. 要怎么做才能使分布更均匀？
   一方面，可以采用多次采样后取均值的方法来缓解随即偏移量带来的采样偏差问题。另一方面，也可以降低随即偏移量的大小，使得采样点间的距离更加均匀，以此来提高采样均匀性。

### 练习 8.3.6

如果我们希望一个序列样本是一个完整的句子，那么这在小批量抽样中会带来怎样的问题？如何解决？

**解答：**
采用小批量采样时，通常会在序列中随机选取采样点位置，从而导致整个句子的完整性被破坏，并且这种切断方式还会破坏句子间的上下文关系，使得模型无法理解句子结构。

解决这种问题，需要采取措施避免句子序列被截断，以及上下文关系被破坏。例如可以采取手动控制采样位置的方式，确定句子边界后，将边界处设置为采样点。或者增加序列长度，确保长句不会被切断，保留更多的完整句子。

## 8.4 循环神经网络 

### 练习 8.4.1

如果我们使用循环神经网络来预测文本序列中的下一个字符，那么任意输出所需的维度是多少？

**解答：**
在循环神经网络中，上一次的输出会被用到本次次的输入中，作为当前时间步的输入与前一时间步的隐藏层进行拼接，因此输出结果的维度需要与隐藏层$H_{t-1}$以及当前输入$X_t$的维度在行上相等，即假设每个字符由一个$d$维向量表示，当前输入$X_t$与隐藏层$H_{t-1}$在行上的维度均应该为$d$，输出结果在行上的维度也应该为$d$。

### 练习 8.4.2

为什么循环神经网络可以基于文本序列中所有先前的词元，在某个时间步表示当前词元的条件概率？

**解答：**
因为语言模型的目标是根据过去和当前的词元来预测出下一个词元，在循环神经网络中，通过引入隐状态的概念，使得RNN模型可以通过隐状态保留了直到当前时间步的历史信息，根据这些过去历史信息和上一个时间步的输出结果，循环神经网络可以预测出当前词元的生成。

### 练习 8.4.3

如果基于一个长序列进行反向传播，梯度会发生什么状况？

**解答：**
基于长序列进行反向传播时，假设序列长度为$T$，则迭代计算$T$个时间步的梯度时，会产生长度为$O(T)$的矩阵乘法链，造成类似多层感知机中层数为$T$的情况，因此当序列长度过长时，会使得梯度值在反向传播进行梯度累乘时放大或缩小梯度值，导致数值不稳定，出现梯度爆炸或者梯度消失等现象。
可采用梯度裁剪的方法，设置阈值$\theta$，当梯度长度超过$\theta$时，通过公式
$$
g \leftarrow min(1, \frac{\theta}{||g||})g
$$
可将梯度长度限制为$\theta$，有效预防梯度爆炸等数值不稳定问题。

### 练习 8.4.4

与本节中描述的语言模型相关的问题有哪些？

**解答：**
循环神经网络常被用来处理序列问题，具体应用场景如下：
- 文本生成：根据给定文本序列内容预测下一个词的生成。
- 文本分类：根据给定序列内容，在最后输出序列类型（情感分析）
- 机器翻译：将给定的文本序列全部输入完成之后，对文本序列进行翻译，转换为另一种语言

## 8.5 循环神经网络的从零开始实现 

### 练习 8.5.1

尝试说明独热编码等价于为每个对象选择不同的嵌入表示。

**解答：**
one-hot编码可以将词表中的每个单词编码为一个独立符号，使得任何词在与词表同样大小的one-hot向量中都具有一个独一无二的维度，实现对不同词的不同嵌入表示。

而嵌入表示建立起一个低维稠密向量空间，也是一种将对象转为低维连续向量的方法，可以根据向量来目标对象间的相似性和差异。因此可将二者视为等价情况，即对于一个N维嵌入向量对表示的N个词，可以将其one-hot编码等价视为嵌入表示中每个词仅有唯一一个元素非零的特殊情况。

### 练习 8.5.2

通过调整超参数（如迭代周期数、隐藏单元数、小批量数据的时间步数、学习率等）来改善困惑度。
1. 困惑度可以降到多少？
2. 用可学习的嵌入表示替换独热编码，是否会带来更好的表现？
3. 如果用H.G.Wells的其他书作为数据集时效果如何，例如[*世界大战*](http://www.gutenberg.org/ebooks/36)？

**解答：**
1. 将小批量数据的时间步数缩小到10后，困惑度由1增大到1.8，模型的性能出现下降，在原有基础上增大时间步后，困惑度降低为1，增大时间步数可以有效降低困惑度。
   尝试增加隐藏单元数至1024，模型困惑度与性能未发生变化，仍为1.0与1.4。增加迭代周期数时效果也是如此。
   缩小学习率为0.01，模型无法在500个epoch中很好训练，困惑度暴增至16.5。增大学习率为10后，模型发生梯度爆炸，训练效果极差。
   ![](../../images/lr10.png)
   多次尝试后发现，困惑度最低降低到1，此时模型可以完美估计下一个词元，性能最佳。
2. 若采用可学习嵌入表示对one-hot编码进行替换，会带来更好的表现，因为可学习的嵌入表示可以将相似近义词映射到相似嵌入向量中，而不是one-hot中相互独立的向量基。
   这种做法可以使得语义信息得到更好的表示，令神经网络能够更好地学习其类别相似性关系。
   One-Hot时结果如下：
   ![](../../images/onehot.png)
   修改为可学习嵌入，设置embedding维度为28：
   ```
   class RNNModel_Embedding(nn.Module):
    def __init__(self, rnn_layer, vocab_size, embedding_dim, **kwargs):
        super(RNNModel_Embedding, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        X = self.embedding(inputs.T.long())
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 全连接层首先将Y的形状改为(时间步数*批量大小,隐藏单元数)
        # 它的输出形状是(时间步数*批量大小,词表大小)。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((
                        self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens), device=device))
   ```
   ![](../../images/embedding.png)
3. 使用《世界大战》作为数据集时，数据集信息如下：
   ![](../../images/read_world.png)
   ![](../../images/read_world_2.png)
   模型的训练效果和预测结果：
   ![](../../images/world.png)
   可以看到，相较于时间机器，更换数据集后，文本数据量明显增加，最终预测结果更好。

### 练习 8.5.3

修改预测函数，例如使用采样，而不是选择最有可能的下一个字符。
1. 会发生什么？
1. 调整模型使之偏向更可能的输出，例如，当$\alpha > 1$，从$q(x_t \mid x_{t-1}, \ldots, x_1) \propto P(x_t \mid x_{t-1}, \ldots, x_1)^\alpha$中采样。

**解答：** 

### 练习 8.5.4

在不裁剪梯度的情况下运行本节中的代码会发生什么？

**解答：**
在不裁剪梯度的情况下，相较于裁剪梯度，模型在训练时的困惑度出现了波动，导致曲线出现不平滑的现象。
不裁剪梯度时，困惑度变化情况如下:
![](../../images/un_grad_clip.png)
裁剪后，困惑度曲线如下:
![](../../images/grad_clip.png)

### 练习 8.5.5

更改顺序划分，使其不会从计算图中分离隐状态。运行时间会有变化吗？困惑度呢？

**解答：**
对顺序划分进行修改，改为先对输入和标签进行设备变换和形状变换，再进行前向计算和反向传播，将隐状态的分离操作放在更新之前，避免在更新中对隐状态进行计算，无需对隐状态进行修改，因此可以避免隐状态从计算图中分离的问题。

```
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """训练网络一个迭代周期（定义见第8章）"""
    metric = d2l.Accumulator(2)  # 训练损失之和,词元数量
    for X, Y in train_iter:
        X, Y = X.to(device), Y.to(device)
        y = Y.T.reshape(-1)
        state = net.begin_state(batch_size=X.shape[0], device=device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward(retain_graph=True)
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward(retain_graph=True)
            grad_clipping(net, 1)
            # 因为已经调用了mean函数
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()
```

修改前：
![](../../images/hidden.png)
修改后：
![](../../images/un_hidden.png)
可以看到，这样的修改加快了词元速度，使得运行时间得到了减少，但困惑度也随之增加。

### 练习 8.5.6

用ReLU替换本节中使用的激活函数，并重复本节中的实验。我们还需要梯度裁剪吗？为什么？

**解答：**

![](../../images/ReLU.png)

使用ReLU激活函数后，无需进行梯度裁剪。因为原始RNN中采用tanh作为激活函数，而tanh会使得梯度在反向传播时不断缩小或放大，导致出现梯度爆炸或梯度消失的现象。
因此需要进行梯度裁剪，避免因过长的求导链出现数值不稳定的现象。
而ReLU作为激活函数时，其梯度在输入为正时恒为1，输入为负时恒为0，因此在反向传播时通常不会出现数值不稳定现象，因此无需进行梯度裁剪。

## 8.6 循环神经网络的简洁实现 

### 练习 8.6.1

尝试使用高级API，能使循环神经网络模型过拟合吗？

**解答：**
在高级API中，针对循环神经网络进行了一系列优化，例如nn.RNN源码中增加了可开启的Dropout模块，以及可选的ReLU激活函数，d2l.train_ch8函数中也默认开启了梯度裁剪，这些措施均可以很好的缓解模型在训练时的过拟合问题，使得模型不会轻易发生梯度爆炸。

但是当学习率设置过大时，例如由1增加到10，过大的学习率会使得模型在优化时无法找到最优值，反而螺旋上升，发生梯度爆炸。
![](../../images/api_overfitting.png)

### 练习 8.6.2

如果在循环神经网络模型中增加隐藏层的数量会发生什么？能使模型正常工作吗？

**解答：**
增加隐藏层的数量会使得模型变得更加复杂，一定程度上可以提高模型预测效果，但过于复杂的模型也会导致出现过拟合现象。
将隐藏层由1增加到3，相较于单层隐藏层，模型的困惑度由1.3降低到1，但是token的生成速度也出现了明显下降。
![](../../images/hidden=3.png)
增加前：
![](../../images/onehot.png)
增加隐藏层为3后：
![](../../images/hidden=3.png)
增加隐藏层数到100时，训练速度出现明显下降，模型也发生过拟合。
![](../../images/hidden=100.png)

### 练习 8.6.3

尝试使用循环神经网络实现8.1节中的自回归模型。

**解答：** 

## 8.7 通过时间反向传播 

### 练习 8.7.1

假设我们拥有一个对称矩阵$\mathbf{M} \in \mathbb{R}^{n \times n}$，其特征值为$\lambda_i$，对应的特征向量是$\mathbf{v}_i$（$i = 1, \ldots, n$）。通常情况下，假设特征值的序列顺序为$|\lambda_i| \geq |\lambda_{i+1}|$。
   1. 证明$\mathbf{M}^k$拥有特征值$\lambda_i^k$。
   1. 证明对于一个随机向量$\mathbf{x} \in \mathbb{R}^n$，$\mathbf{M}^k \mathbf{x}$将有较高概率与$\mathbf{M}$的特征向量$\mathbf{v}_1$在一条直线上。形式化这个证明过程。
   1. 上述结果对于循环神经网络中的梯度意味着什么？

**解答：**
1. 对于对称矩阵$M$，其特征值为$\lambda_i$，且拥有特征向量$v_i$，因此该对称矩阵满足：
   $$M \cdot v_i=\lambda_i \cdot v_i$$
   将上述公式两边同时乘上对称矩阵$M$后，公式满足：
   $$M \cdot M \cdot v_i = M \cdot \lambda_i \cdot v_i$$
   $$M^2 \cdot v_i = \lambda_i \cdot (M \cdot v_i) = {\lambda_i}^2 \cdot v_i$$
   重复上述步骤，对于$M^k$，存在：
   $$M^k \cdot v_i = {\lambda_i}^k \cdot v_i$$
   证得矩阵$M^k$具有特征值${\lambda_i}^k$

2. 对于随机向量$x \in R^n$，可将其分解到特征向量$V$所在的向量空间中，即对于分解系数$\alpha_i$，存在分解式：
   $$x = \alpha_1 v1+\alpha_2 v2+ ... + \alpha_n v_n = \sum_{i=1}^n \alpha_i v_i$$
   将上述公式两边同乘$M^k$，存在：
   $$\begin{align}
    M^k \cdot x
    &= \lambda_i^k \cdot x \\
    &= \lambda_i^k \cdot \sum_{i=1}^n \alpha_i v_i \\
    &= \sum_{i=1}^n \lambda_i^k \alpha_i v_i
    \end{align}$$
   又因为$M$的特征值$\lambda_i$满足$|\lambda_i| \geq |\lambda_{i+1}|$，因此$lambda_1^k >> lambda_i$，即$\lambda_1^k$的权重最大。
   因此$M^k \cdot x \approx \lambda_1^k \alpha_1 v_1$，即存在较高概率与特征向量$v_1$在一条直线上。

3. 可将上述公式中的$M^k$理解为权重矩阵，当循环神经网络中，权重矩阵的特征向量与输入向量满足：
   $$M \cdot x = \lambda_1 \alpha_1 v_1$$
   其中$\alpha$根据$x$在特征向量$v$上投影的大小所决定，$\alpha$的变化会导致权重矩阵的变化，进而使得其在反向传播时的梯度受到影响。
   这种变化会随着序列长度的增加不断加剧，使得梯度不断放大或缩小，导致梯度爆炸或梯度消失的问题。

### 练习 8.7.2

除了梯度截断，还有其他方法来应对循环神经网络中的梯度爆炸吗？

**解答：**

除了进行梯度截断，还可采用下述方法解决循环神经网络中的梯度爆炸：
- 长短期记忆网络：可以通过引入门控机制或注意力机制的方法，减少梯度异常发生的概率，对循环神经网络进行改进。
- 对权重进行正则化，通过添加L1或L2正则项减少梯度爆炸发生的概率
- 修改激活函数，将激活函数更换为ReLU，可以有效避免梯度爆炸问题